
Импорт библиотек

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Анализ данных

In [64]:
dt = pd.read_html('/content/Выгрузка 9 апреля.xls')

In [65]:
df = dt[1]
df.head()

,№ п/п,№документа,Датарегистрации,Исх. №Дата,Адресат,Автор,Краткоесодержание,Примечание
0,1,WSR/1-838/2019,09.04.2019,208-40908.04.2019,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...,NaN
1,2,WSR/1-837/2019,09.04.2019,05-33809.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,NaN
2,3,WSR/1-835/2019,09.04.2019,И10-14/313709.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,NaN
3,4,WSR/1-834/2019,09.04.2019,Д2/7121-ИС03.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,NaN
4,5,WSR/1-833/2019,09.04.2019,15-20/06-91108.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...,NaN


Информация

In [66]:
df.describe()

,№ п/п
count,4967.000000
mean,2484.000000
std,1433.993724
min,1.000000
25%,1242.500000
50%,2484.000000
75%,3725.500000
max,4967.000000


In [67]:
df.shape

(4967, 8)

In [68]:
df.info

<bound method DataFrame.info of       № п/п  ... Примечание
0         1  ...        NaN
1         2  ...        NaN
2         3  ...        NaN
3         4  ...        NaN
4         5  ...        NaN
...     ...  ...        ...
4962   4963  ...        NaN
4963   4964  ...        NaN
4964   4965  ...        NaN
4965   4966  ...        NaN
4966   4967  ...        NaN

[4967 rows x 8 columns]>

Предобработка данных и выделение значимых атрибутов

In [69]:
dt[0].T[0]
dt[0].T[1].unique().tolist()
dt[0].T[2].unique().tolist()
df = dt[1]

In [70]:
df = dt[1].drop(['№ п/п', '№документа', 'Исх. №Дата', 'Примечание'], axis=1)
df.head(2)

,Датарегистрации,Адресат,Автор,Краткоесодержание
0,09.04.2019,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...
1,09.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...


Регулярные выражение

In [71]:
import re

In [72]:
adres = []
c = 0
reg_fio = r'([А-Я]\.[А-Я]\.\s?[А-Я][а-я]{1,20}|[А-Я][а-я]{1,20}\s?[А-Я]\.[А-Я]\.)'
for i in df.Адресат:
  result = re.findall(reg_fio, i)[-1]
  adres.append(result.replace(' ',''))

df['Адресат(-ы)'] = adres
df.tail(2)

,Датарегистрации,Адресат,Автор,Краткоесодержание,Адресат(-ы)
4965,01.11.2017,Уразов Р.Н.,Огородова Л.М. (Министерство образования и нау...,Протокол заседания рабочей группы приоритетног...,УразовР.Н.
4966,NaN,Уразов Р.Н.,Асаул Н.А. (Министерство транспорта Российской...,Участие в Чемпионате WorldSkills Hi-Tech Екате...,УразовР.Н.


Дополнение недостающими данными

In [73]:
df['Адресат(-ы)'].unique().shape

(81,)

In [74]:
df.Датарегистрации = pd.to_datetime(df.Датарегистрации)
df.Датарегистрации.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


count                    4965
unique                    372
top       2017-11-29 00:00:00
freq                       42
first     2017-01-11 00:00:00
last      2019-12-03 00:00:00
Name: Датарегистрации, dtype: object

In [75]:
df['Агентсво'] = df.Автор.apply(lambda x: x.split('(')[1].replace(')',''))

In [76]:
df.head(2)

,Датарегистрации,Адресат,Автор,Краткоесодержание,Адресат(-ы),Агентсво
0,2019-09-04,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...,УразовР.Н.,АО «Интерфакс»
1,2019-09-04,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,УразовР.Н.,Министерство просвещения Российской Федерации


Переписать список откуда и куда, сравнивать строки

In [77]:
agents = []
target = ['Министерств','Правительств','Федеральн','Администр']
for i in df.Агентсво:
  br = 0
  for t in target:
    if br:
      break
    if t in i:
      br = 1
      agents.append(1)
  if not(br):
    agents.append(0)

In [78]:
df['Тип_агентсва'] = agents
df.head()

,Датарегистрации,Адресат,Автор,Краткоесодержание,Адресат(-ы),Агентсво,Тип_агентсва
0,2019-09-04,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...,УразовР.Н.,АО «Интерфакс»,0
1,2019-09-04,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,УразовР.Н.,Министерство просвещения Российской Федерации,1
2,2019-09-04,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,УразовР.Н.,Администрация Кемеровской области,1
3,2019-09-04,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,ИванюкЛ.А.,Министерство транспорта Российской Федерации,1
4,2019-09-04,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...,УразовР.Н.,Ставропольский государственный аграрный универ...,0


# Предварительная обработка текста

In [79]:
!pip install pymorphy2

In [80]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Удаление стоп-слов

In [81]:
all_stopwords = stopwords.words("russian")

In [82]:
%%time
list2 = []
for j in df.Краткоесодержание:
  list1 = []
  for i in j.split(" "):
    if i not in all_stopwords:
      list1.append(i)
  
  list2.append((" ").join(list1))

CPU times: user 166 ms, sys: 3.27 ms, total: 169 ms
Wall time: 172 ms


Всё фурычит :)))

In [83]:
from nltk.stem.snowball import SnowballStemmer

In [84]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
def stm(doc):
    tokens = []
    for token in doc.split():
      token = stemmer.stem(token)
      tokens.append(token)
    if len(tokens) > 0:
        return (" ").join(tokens)
    return None

In [85]:
df.Краткоесодержание = df.Краткоесодержание.apply(stm)

In [86]:
df.Краткоесодержание[10]

'вх - об участ в заочн заседан совет по профквалификац в машиностроение, 8 апр 2019'

#Разработка бота

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [88]:
df.head()

,Датарегистрации,Адресат,Автор,Краткоесодержание,Адресат(-ы),Агентсво,Тип_агентсва
0,2019-09-04,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),"вх - о возможн использован сетев издан ""информ...",УразовР.Н.,АО «Интерфакс»,0
1,2019-09-04,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,вх - об участ в совещан по вопрос доработк фэ ...,УразовР.Н.,Министерство просвещения Российской Федерации,1
2,2019-09-04,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),вх - о командирован саликов к. и глушк д. для ...,УразовР.Н.,Администрация Кемеровской области,1
3,2019-09-04,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,вх - об участ в чемпионат мир по профессиональ...,ИванюкЛ.А.,Министерство транспорта Российской Федерации,1
4,2019-09-04,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,вх - об участ в церемон открыт вузовск чемпион...,УразовР.Н.,Ставропольский государственный аграрный универ...,0


In [89]:
X = df.Краткоесодержание
y = df.Адресат

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train = X_train.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [91]:
vectorizer = TfidfVectorizer(max_features = 10000)

In [92]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [93]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score 

In [94]:
%%time
model_SVC = SVC(C = 1, kernel = 'linear')
model_SVC.fit(X_train, y_train)
res = model_SVC.predict(X_test)

CPU times: user 5.79 s, sys: 698 µs, total: 5.8 s
Wall time: 5.82 s


In [95]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

from sklearn import metrics
print("Точность:", metrics.accuracy_score(y_test, y_pred))

Точность: 0.6830985915492958


In [96]:
def adres():
    classificationResult = clf.predict(vectorizer.transform([request1]))
    return classificationResult[0]

request = input('Документация: Определить адресата введите /adresat\nОписание естественным языком введите /text\nДокументация /help\nВведите - ')
if request == "/adresat" or request == "/text":
    request1 = input("Введите документ - ")
    print("Адресат - " + adres())
elif request == "/help":
    print("Документация: Определить адресата введите /adresat\nОписание естественным языком введите /text")
else:
    print("А я тебя не понял :D")

Документация: Определить адресата введите /adresat
Описание естественным языком введите /text
Документация /help
Введите - 1
А я тебя не понял :D


In [97]:
def adres():
    classificationResult = clf.predict(vectorizer.transform([request1]))
    return classificationResult[0]

request = input('Документация: Определить адресата введите /adresat\nОписание естественным языком введите /text\nДокументация /help\nВведите - ')
if request == "/adresat" or request == "/text":
    request1 = input("Введите документ - ")
    print("Адресат - " + adres())
elif request == "/help":
    print("Документация: Определить адресата введите /adresat\nОписание естественным языком введите /text")
else:
    print("А я тебя не понял :D")

Документация: Определить адресата введите /adresat
Описание естественным языком введите /text
Документация /help
Введите - 1
А я тебя не понял :D


In [98]:
def adres():
    classificationResult = clf.predict(vectorizer.transform([request1]))
    return classificationResult[0]

request = input('Документация: Определить адресата введите /adresat\nОписание естественным языком введите /text\nДокументация /help\nВведите - ')
if request == "/adresat" or request == "/text":
    request1 = input("Введите документ - ")
    print("Адресат - " + adres())
elif request == "/help":
    print("Документация: Определить адресата введите /adresat\nОписание естественным языком введите /text")
else:
    print("А я тебя не понял :D")

Документация: Определить адресата введите /adresat
Описание естественным языком введите /text
Документация /help
Введите - 1
А я тебя не понял :D


#Разработка API

In [99]:
import pickle

Скачал обученную модель

In [102]:
pickle.dump(clf, open('rf_cls', 'wb'))